In [ ]:
import holoviews as hv
hv.extension('bokeh')
hv.opts.defaults(hv.opts.Curve(width=500), 
                 hv.opts.Histogram(width=500))

In [ ]:
import numpy as np
import scipy.stats

# Inferencia estadística con tests no-paramétricos


## Introducción

En esta lección revisaremos metodologías y tests estadísticos de la categoría no parámetrica, es decir que no presuponen una distribución en particular para las observaciones

Esto es contrario a las técnicas parámetricas que vimos en lecciones anteriores. Por ejemplo, un t-test parte del supuesto de que nuestros datos se distribuyen normal. Con este supuesto somos capaces de obtener una distribución para el estadístico de prueba ante la hipótesis nula. Esto a su vez nos permite calcular p-values y finalmente concluir si la hipótesis nula puede ser rechazada. 

Si los supuestos que hicimos sobre la población son incorrectos entonces las conclusiones del método parámetrico pierden validez. Sin embargo en la práctica los tests parámetricos funcionan bien ante "ligeras" desviaciones de los supuestos

Como recomendación general considere

- **Comprobar** los supuestos antes de hacer el análisis con métodos parámetricos
- Si los supuestos no se cumplen una opción es usar **inferencia no paramétrica**

A continuación veremos tests que nos permiten comprobar si nuestros datos siguen una distribución en particular. Luego veremos las alternativas no-parámetricas para los tests paramétricos más comunes. Finalmente veremos como usar la técnica de *bootstrap resampling* para estimar intervalos de confianza.

## Evaluar la normalidad de una distribución

La normalidad es un supuesto en muchos procedimientos parámetricos, por ejemplo: t-test, ANOVA, regresion lineal

Tengamos en cuanta las características principales de la distribución normal

- Unimodal con media igual a su moda
- Simétrica en torno a la media
- Concentrada en torno a la media: ~68% a $\pm \sigma$, ~95% a $\pm 2\sigma$, ...

Si tenemos suficientes muestras podriamos observar el histograma y comprobar visualmente si esto se cumple. A modo de ejemplo observe las siguientes distribuciones y discuta

- ¿Cuáles son no-normales? 
- ¿Cuál o cuáles características de la distribución normal no se están cumpliendo en cada caso?

In [ ]:
data1 = scipy.stats.norm(loc=4, scale=2).rvs(1000) # Normal
data2 = scipy.stats.gamma(a=2, scale=1).rvs(1000) # Gamma
data3 = scipy.stats.uniform(loc=-2, scale=10).rvs(1000) #Uniforme

In [ ]:
p = []
for i, data in enumerate([data1, data2, data3]):
    bins, edges = np.histogram(data, bins=20, range=(-4, 10), density=True)
    p.append(hv.Histogram((edges, bins), kdims='Datos', vdims='Densidad'))
hv.Layout(p).cols(3).opts(hv.opts.Histogram(width=250, height=250))

La visualización mediante histograma es útil como un paso exploratorio. Sin embargo podemos comprobar normalidad de forma más rigurosa usando un test

Por ejemplo se puede formular un test sencillo basándonos en en que la distribución normal debiese tener una simetría (tercer momento) igual a cero. 

Con `scipy` podemos calcular la simentría usando la función `scipy.stats.skew`. Además el test anteriormente mencionado está implementado en [`scipy.stats.skewtest`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.skewtest.html)

Por ejemplo para las tres distribuciones anteriores tenemos que

In [ ]:
for i, data in enumerate([data1, data2, data3]):
    display(scipy.stats.skewtest(data))


- Para la muestra `data2` rechazamos la hipótesis de que la simetría corresponda al de una distribución normal
- Para las muestras `data1` y `data3` no podemos rechazar la hipótesis nula


Podemos complementar lo anterior con un test sobre la kurtosis, el cuarto momento de la distribución. En la definición por defecto de la kurtosis (Fisher) su valor debiese ser cero si la distribución subyacente es normal

Con `scipy` podemos calcular la kurtosis con la función [`scipy.stats.kurtosis`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kurtosis.html). Además está implementado un test para la hipótesis de que la kurtosis corresponde al de una distribución normal: [`scipy.stats.kurtosistest`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kurtosistest.html)

Para las tres muestras anteriores tenemos

In [ ]:
for i, data in enumerate([data1, data2, data3]):
    display(scipy.stats.kurtosistest(data))

- Para la muestra `data2` rechazamos las hipótesis de que la kurtosis y simetría correspondan al de una distribución normal
- Para la muestra `data3` rechazamos la hipótesis de que la kurtosis corresponde al de una distribución normal
- Para la muestra `data1` no podemos rechazar la hipótesis nula

Estos tests son simples pero particulares para la distribución normal. A continuación veremos formas más generales para comparar muestras con distribuciones teóricas

## Comparando distribuciones empíricas y teóricas

En esta sección revisaremos algunos métodos para evaluar si la distribución empírica de nuestras datos sigue alguna distribución teórica en particular. 

En los ejemplos usaremos una distribución teórica normal sin embargo puede usarse cualquier otra distribución bajo ciertas condiciones que se detallan a continuación

### Test de Kolmogorov-Smirnov (KS)

El test de KS es un test no-paraḿetrico que examina la distancia entre las funciones de distribución acumulada (CDF) entre dos muestras o entre una muestra y una distribución teórica. El test sólo se puede usar para distribuciones univariadas y continuas

Nos concentraremos en el caso de una muestra, es decir cuando queremos verificar si nuestros datos siguen una distribución teórica en particular

El estadístico de test para KS es

$$
D_n = \sup_x |F_n(x) - F(x)|
$$

donde $F(x) = \int_{-\infty}^x f(x) dx = P(X<x)$ es la CDF teórica y 

$$
F_n(x) = \frac{1}{N} \sum_{i=1}^N \mathbb{1}[X_i<x]
$$

es la distribución acumulada empírica (ECDF)

Primero veamos como se ve la ECDF de las tres muestras anteriores contra la CDF de una normal estándar

In [ ]:
def ecdf(data):
    x = np.sort(data)
    n = x.size
    y = np.arange(1, n+1) / n
    return(x, y)

In [ ]:
p = []
for i, data in enumerate([data1, data2, data3]):
    # Normalizar
    data = ((data - np.mean(data))/np.std(data)).copy() 
    # Calcular ECDF
    x, y = ecdf(data)
    # CDF standard normal
    x_t = np.linspace(-2, 3, num=100)
    y_t = scipy.stats.norm.cdf(x_t)    
    p.append(hv.Curve((x, y), 'Datos', 'Distribución acumulada', label='ECDF') * hv.Curve((x_t, y_t), label='CDF normal'))
hv.Layout(p).cols(3).opts(hv.opts.Curve(width=250, height=250, alpha=0.75), 
                          hv.opts.Overlay(legend_position='bottom_right'))

Según la definición anterior $D_n$ es el mayor de los errores absolutos entre la CDF teórica y empírica

Las hipótesis de la prueba KS de una muestra son

- $\mathcal{H}_0$: Los datos siguen la distribución teórica especificada
- $\mathcal{H}_A$: Los datos no siguen la distribución teórica especificada

Si $D_n$ es menor que el valor crítico no podemos rechazar $H_0$

:::{tabbed} Python

La librería `scipy` tiene implementado el test de KS. Sus argumentos son

```python
scipy.stats.kstest(rvs, # Arreglo de muestras o función rvs de scipy.stats
                   cdf, # String o función cdf de scipy.stats para test de una muestra
                   args=(), # Argumentos que se pasan a rvs y cdf si son funciones
                   N=20, # Número de muestras generadas si rvs es una función
                   alternative='two-sided', # String que define la hipótesis alternativa
                   ...
                  )
```

La función retorna una estructura con el valor del $D_n$ y el p-value

:::

:::{tabbed} R

El paquete `stats` tiene implementado el test de KS. Sus argumentos son

```R
ks.test(x, # Vector con datos numéricos
        y, # String o función cdf para test de una muestra
        alternative = "two.sided", # String que define la hipótesis alternativa
        exact = NULL # Indica si el p-value se calcula de forma exacta
       )
```

La función returna una objeto `htest` con los componentes `statistic` para $D_n$ y `p.value` para el p-value

:::





En el siguiente ejemplo se usa `scipy.stats.kstest` sobre las tres muestras anteriores. Se compara cada muestra con la CDF de una distribución normal cuya media y desviación estándar son equivalentes a los de la muestra

In [ ]:
for i, data in enumerate([data1, data2, data3]):
    # CDF    
    theorical_cdf = scipy.stats.norm(loc=np.mean(data), scale=np.std(data)).cdf
    # Realizar test de KS
    display(scipy.stats.kstest(data, theorical_cdf)) 

Asumiendo un $\alpha=0.01$ 

- Rechazamos la hipótesis nula de que la distribución es normal para las muestras `data2` y `data3`
- No podemos rechazar la hipótesis nula en el caso de  `data1`


El test de Kolmogorov-Smirnov es ampliamente usado pero tiene algunas debilidades

1. Solo sirve para distribuciones continuas y univariadas
1. Es más sensible en el centro de la distribución que en las colas de la distribución
1. Se debe especificar completamente la distribución teórica 

Respecto a la primera debilidad en el caso de querer comparar distribuciones discretas la alternativa es el test no paramétrico de chi cuadrado

Respecto del segundo punto se pueden considar los siguientes tsts que dan mayor ponderación a las colas:

- [Test de Anderson-Darling](https://www.itl.nist.gov/div898/handbook/prc/section2/prc213.htm), con implementación en [`scipy.stats.anderson`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.anderson.html)  (normal, exponencial y gumbel)
- [Test de Shapiro-Wilks](https://www.itl.nist.gov/div898/handbook/prc/section2/prc213.htm) test con implementación en [`scipy.stats.shapiro`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html) (solo normal)

Finalmente si quiere probar normalidad pero no se sabe la media o la varianza se puede usar el test de [Lilliefors](https://www.statsmodels.org/0.6.1/generated/statsmodels.stats.diagnostic.lillifors.html?highlight=lilliefors)


### Probability plots (PP)

Otra opción para comparar distribuciones de forma gráfica son los *probability plots*

1. Se calculan los cuantiles (qq) o percentiles (pp) de dos muestras **o** de una muestra y una distribución teórica
1. Se grafica uno en función del otro
1. Mientras más se parezca el resultado a una linea recta más similares son las distribuciones

Podemos usar [`scipy.stats.probplot`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.probplot.html#scipy.stats.probplot) para comparar una muestra con una distribución teórica 

Nuevamente para nuestras tres muestras con una distribución normal. Dado que la prueba es basada en rangos no necesitamos entregar parámetros de localización (media) o escala (varianza). Cualquier parámetro de otro tipo (e.g. shape) se debe indicar


In [ ]:
p = []
for i, data in enumerate([data1, data2, data3]):
    (osm, osr), (w, b, r2) = scipy.stats.probplot(data, dist="norm", fit=True)
    x = np.arange(-3, 4)
    y = x*w + b
    p.append(hv.Curve((osm, osr), 'Theorical', 'Empirical', label='Data') * hv.Curve((x, y), label='Ideal'))
hv.Layout(p).cols(3).opts(hv.opts.Curve(width=250, height=250, alpha=0.75), 
                          hv.opts.Overlay(legend_position='top_left'))

## Elementos de inferencia no paramétrica

Inferencia se refiere a estimar parámetros o probar hipótesis sobre una población en base a muestras. En inferencia **no parámetrica** usamos estadísticos cuya distribución **no depende de supuestos sobre la distribución de la población**

En resumen

:::{important}

No paramétrico $=$ No asumimos **distribución específica** para la población 

:::

Sin embargo

:::{warning}

No paramétrico $\neq$ Libre de supuestos o Libre de (hiper)parámetros

:::

Por ejemplo un supuesto que muchos tests no parámetricos tienen en común es que las muestras son independientes. Otro supuesto o requisito común es que se cumpla un cierto número mínimo de muestras


¿Por que no usar siempre tests no paramétrics? En general los tests no parámetricos tienen un menor poder estadístico (sensibilidad) que sus contrapartes paramétricas (siempre y cuando sus supuestos se cumplen)

A continuación veremos algunos conceptos fundamentales de inferencia no parámetrica para luego revisar algunos tests no paramétricos que son alternativas a los tests vistos en lecciones anteriores

### Función cuantil y estadístico de orden


La CDF se define como

$$
P(X<x) = F(x) = p,
$$

donde $p\in [0, 1]$

En algunos casos nos interesa saber que valor $x$ corresponde a un valor dado $p$. Esto se hace con la función cuantil o CDF inversa

$$
Q(p) = \inf_{x\in \mathbb{R}} p \leq F(x)
$$

En scipy podemos calcular $x$ a partir de $p$ usando el atributo `ppf` (percent-point function) de una distribución de `scipy.stats`, por ejemplo

In [ ]:
# Que valor de x corresponde a un 50% de la distribución normal?
scipy.stats.norm.ppf(0.5)

In [ ]:
# y a un 99%?
scipy.stats.norm.ppf(0.98)

Sea ${X_1, X_2, X_3, \ldots, X_N}$ un conjunto de VAs iid y ${x_1, x_2, x_3, \ldots, x_N}$ una muestra aleatoria

Si asumimos que la población tiene una CDF continua (no hay dos VA con el mismo valor) entonces podemos hacer un ordenamiento único

$$
x_{(1)} < x_{(2)} < x_{(3)} < \ldots < x_{(N)} 
$$

que se llaman colectivamente el estadístico de orden de la muestra aleatoria. El estadístico específico $x_{(r)}$ se llama el estadístico de orden r

### Distribución acumulada empírica (ECDF)

Con el estadístico de orden se construye la ECDF que vimos anteriormente

$$
F_N(x) = \begin{cases} 0 & x < x_{(1)} \\ 
\frac{i}{N} & x_{(i-1)} \leq x < x_{(i)},  i=2,3,\ldots,N \\ 
1 & x_{(N)} < x\end{cases}
$$


In [ ]:
# Implementación simple en Python
def ecdf(data):
    x = np.sort(data)
    n = x.size
    y = np.arange(1, n+1) / n
    return(x, y)

Notar que el ECDF es un estimador insesgado

$$
\mathbb{E}[F_N(x)] = F(x)
$$

y su varianza tiene a cero con $N$

$$
\text{Var}[F_N(x)] = \frac{1}{N} F(x) ( 1- F(x))
$$

Adicionalmente la ECDF

- es un estimador consistente de $F(x)$ (converge en probabilidad) 
- (estandarizada) es un asintoticamente normal estándar


### Función cuantil empírica

También podemos usar el estadístico de orden para construir la función cuantil empírica

$$
Q_N(u) = \begin{cases}
x_{(1)} &  0 < u \leq \frac{1}{N} \\
x_{(2)} & \frac{1}{N} < u  \leq\frac{2}{N} \\
\vdots & \vdots \\
x_{(N)} & \frac{(N-1)}{N} < u  \leq 1 \\
\end{cases}
$$

En Python podemos usar `np.quantile` y `np.percentile` para calcular cuantiles y percentiles de una muestra, respectivamente

In [ ]:
data = np.random.randn(1000)
np.quantile(data, 0.5) # NUMEROS ENTRE 0 Y 1 

In [ ]:
np.percentile(data, 50)

### Aplicaciones de los estadísticos de orden

Dos estadísticos muy usuales que vienen de los estadísticos de orden son

- El rango: $x_{(N)} - x_{(1)}$
- La mediana: $x_{(n/2)}$ (n par)

Los estadísticos de orden suelen ser más robustos que sus contrapartes en la presencia de *outliers*

En Python podemos calcular la mediana con `np.median`

In [ ]:
x = np.random.randn(10) # media cero y desviación estándar uno
x[9] = 100 # Inserto un outlier

In [ ]:
# Le media es:
np.mean(x)

In [ ]:
# En cambio la mediana:
np.median(x)

es claramente menos afectada por el outlier

Otro estadístico muy usado es el rango intercuartil, que corresponde a la diferencia entre cuantil 0.75 y 0.25

In [ ]:
np.subtract.reduce(np.quantile(x, [0.75, 0.25]))

este es una alternativa robusta ante outliers para la desviación estándar

In [ ]:
np.std(x)

## Tests no paramétricos

El test de KS que vimos antes es un ejemplo de test no paramétrico

Las propiedades de la ECDF nos permiten construir distribuciones nulas sin necesidad de asumir una distribución para la población

Existen muchos tests no-paramétricos, a continuación revisaremos algunos de ellos en detalle

### Test de Mann-Whitney U

Es una prueba no-paramétrica para comparar dos **muestras independientes**

- Ejemplo de muestras independientes: notas de niños y niñas en una prueba
- Ejemplo de muestras dependientes: notas de un mismo curso en pruebas consecutivas

El objetivo del test es probar si las muestras independientes provienen de una misma población en función de su tendencia central

Esto es similar a lo que realiza el test parámetric *t-test* en donde se comparan dos medias $\mu_0$ y $\mu_1$ asumiendo que la distribución subyacente es normal

El test de Mann-Whitney U no supone distribución para la población y algunos lo interpretan como una comparación entre medianas. La hipótesis nula es que no hay diferencia entre las distribuciones muestrales

El algoritmo del test es

1. Ordenar las observaciones de ambas muestras juntas
1. Sumar los rangos de la primera muestra $R_1$. $R_2$ queda definida ya que $R_1 + R_2 = \frac{N(N+1)}{2}$
1. Se calcula el estadístico $U_1 = R_1 - \frac{N_1(N_1+1)}{2}$, donde $N_1$ es el número de observaciones de la primera muestra. Por simetría $U_2$ queda inmediatamente especificado
1. Se usa $U = \min(U_1, U_2)$ para calcular el p-value


:::{admonition} Intuición

La prueba mezcla y ordena las observaciones de ambas muestras. Intutivamente, si las muestras son similares entonces sus observaciones deberían mezclarse en el ordenamiento. Si en cambio se clusterizan entonces las distribuciones son distintas

:::


Este test está implementado en 

```python

scipy.stats.mannwhitneyu(x, # Muestra 1
                         y, # Muestra 2
                         alternative=None # usar 'two-sided' para test de dos colas
                         ...
                        )

```

La función retorna el estadístico $U$ (`alternative=None`) y el p-value. 

:::{warning}

El p-value retornado sólo es valido si los conjuntos tienen más de 20 muestras. Si los conjuntos tienen menos de 20 muestras se puede consultar una [tabla](https://math.usask.ca/~laverty/S245/Tables/wmw.pdf) del valor $U_{critico}^\alpha$. Luego si  $U < U_{critico}^\alpha$ entonces se rechaza la hipótesis nula al $1-\alpha$ % de confianza

:::

### Test de los rangos con signo de Wilcoxon

Si las muestras que se quieren comparar son **dependientes** no se puede usar Mann Whitney U. En su lugar se puede usar el test de rango con signo de Wilcoxon

Este test funciona como alternativa no-parámetrica al t-test pareado en caso de que los datos violen el supuesto de normalidad

Un test pareado se usa para probar si hubo diferencia significativa entre el "antes" y el "despues" de aplicar cierto tratamiento o intervención. 

Los supuestos de este test son

- Los datos son continuos
- Los datos son pareados y vienen de la misma población
- Los pares son independientes y se escogen aleatoriamente



Sea $x_{11}, x_{12}, \ldots, x_{1N}$ y sus pares $x_{21}, x_{22}, \ldots, x_{2N}$. La hipótesis nula del test es que la mediana de las diferencias entre pares de observaciones es nula. 

El algoritmo del test de Wilcoxon es

1. Se calculan las diferencias $z_i = x_{2i} - x_{1i}$
1. Se ordenan los valores absolutos de las diferencias $|z|_{(1)}, |z|_{(2)}, \ldots, |z|_{(N)}$ y se reserva el signo de cada diferencia 
1. Se suman los rangos de las diferencias con signo positivo: $W^{+}$
1. Se suman los rangos de las diferencias con signo negativo: $W^{-}$
1. El estadístico de prueba es $\min (W^{+}, W^{-})$ sobre el dataset reducido (se eliminan los $z_{(i)}=0$)

:::{admonition} Intuición

Si la hipótesis nula es cierta, esperariamos que $W^{-}$ y $W^{+}$ sean similares. Si esto no se cumple se esperaría que $W^{+}$ fuera mayor que $W^{-}$

:::



El test está implementado en

```python
scipy.stats.wilcoxon(x, # El primer set de muestras. Si y=None entonces x debe ser igual a la diferencia z
                     y=None, # El segundo set de muestras
                     alternative='two-sided', # Para un test de dos colas
                     mode='auto', 
                     ...
                    )

```

El argumento 

- `mode=approx` indica si se ocupa una aproximación normal para calcular el p-value
- `mode=exact` indica si se obtiene el valor de una tabla. Sólo valido para menos de 25 muestras
- `mode=auto` Usa `approx` para más de 25 muestras y `exact` para menos de 25 


### Test de Kruskall-Wallis: Alternativa a one-way ANOVA

Su objetivo es probar que existen diferencias significativas en una variable dependiente continua en función de una variable independiente categórica (2 o más grupos) 

Está implementado en [`scipy.stats.kruskal`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kruskal.html)



### Kendal $\tau$: Alternativa al coeficiente de correlación de Pearson

Su objetivo es probar si dos variables son estadísticamente dependientes

Está implementando en [`scipy.stats.kendalltau`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kendalltau.html)
 

## *Bootstrap* no paramétrico

*Bootstrap* es una familia de técnicas de remuestreo que permiten obtener distribuciones muestrales para estimadores estadísticos. Las distribuciones muestrales pueden utilizarse para calcular intervalos de confianza o realizar tests de hipótesis. En este sentido *bootstrap* es una alternativa a los métodos tradicionales para hacer test de hipótesis basados en derivaciones algebraicas. 

En esta lección estudiaremos el algoritmo de *bootstrap* original propuesto por [Bradly Efron](https://en.wikipedia.org/wiki/Bradley_Efron) en 1979. Este algoritmo se basa en la técnica de muestreo con reemplazo. 

Como veremos más adelante las principales ventajas de *bootstrap* son que 

- no requiere hacer supuestos sobre la distribución de la población 
- puede aplicarse para prácticamente cualquier estadístico muestral. 

La desventaja es que es relativamente más costoso (computacionalmente), sin embargo esta desventaja es cada vez menos relevante con la alta disponibilidad de capacidad de cómputo


### Muestreo con reemplazo

El siguiente diagrama ejemplifica como opera el muestreo con reemplazo. Para un dataset con $N$ muestras se generan $M$ copias todas de tamaño $N$. En las copias algunas de las observaciones originales no aparecen mientras que otras pueden aparecer más de una vez.


<img src="img/bootstrap_diagram.png" width="500">

El objetivo de esto es simular el proceso que generó la muestra original a partir de la población. El muestreo con reeplazo hace que los datasets generados mantengan las propiedades del dataset original. El único supuesto para que esto se cumpla es que las muestras deben ser *i.i.d.*


### Algoritmo e implementación de bootstrap

El algoritmo de bootstrap para un dataset $x=(x_1, x_2, \ldots, x_N)$ con $N$ observaciones y un estadístico $T(\cdot)$ sería

Repetir $M$ veces

1. Crear un nuevo conjunto $\hat x$ remuestreando $x$ aleatoriamente con reemplazo
1. Calcular $T(\hat x)$ y guardar el resultado 

El resultado es un arreglo de tamaño $M$ que corresponde a la distribución muestral del estadístico $T(\cdot)$

Podemos implementar *bootstrap* usando la función `sklearn.utils.resample` o la función `np.random.choice`. A continuación utilizaremos la primera

In [ ]:
from sklearn.utils import resample

def bootstrap_resampling(data):
    return resample(data, replace=True, n_samples=len(data))

En el siguiente ejemplo se generan 100 muestras sintéticas de una distribución normal con media uno y desviación estándar uno

Para probar nuestra rutina de bootstrap calculemos la distribución muestral de 

$$
T(x) = \frac{1}{N} \sum_{i=1}^N x_i = \bar x
$$

a partir de los datos utilizando *bootstrap*. 

El dataset se remuestrea 10,000 veces, en gneral mientras más veces remuestremos mejor será la resolución de nuestra distribución muestral.

In [ ]:
np.random.seed(12345)
# Datos simulados
data = scipy.stats.norm(loc=1, scale=1).rvs(100)
# Estadístico de prueba
test_statistic = lambda data: np.mean(data)
# Distribución empírica con booststrap
bootstrapped_statistic = [test_statistic(bootstrap_resampling(data)) for t in range(10000)]
# Histograma
bins, edges = np.histogram(bootstrapped_statistic, bins=20, density=True)

Para este caso particular sabemos que $T(x)$ es el estimador de máxima verosimilitud de la media de una distribución normal. Además conocemos su distribución muestral asintótica

In [ ]:
# Distribución muestral asintótica del MLE de la media de la distribución gaussiana
asymptotic_distribution = scipy.stats.norm(loc=np.mean(data), scale=1/np.sqrt(len(data)))
x = np.linspace(*asymptotic_distribution.ppf([0.001, 0.999]), num=200)
px = asymptotic_distribution.pdf(x)

Como muestra el siguiente ejemplo la distribución muestral obtenida con bootstrap coincide con la distribución muestral asintótica 

In [ ]:
p1 = hv.Histogram((edges, bins), kdims='x', vdims='Density', label='Bootstrap')
p2 = hv.Curve((x, px), label='Asintótica').opts(color='k')
p1 * p2

Supongamos ahora que en lugar de la media nos interesa la distribución del siguiente estimador (totalmente arbitrario)

$$
T(x) = \frac{1}{N} \sum_{i=1}^N x_i^2 (x_i - 1)
$$

En este tipo de casos, donde no tenemos una distribución asintótica conocida, es donde bootstrap resulta sumamente valioso

Aplicando el mismo procedimiento anterior al nuevo estadístico

In [ ]:
test_statistic = lambda data: np.mean(data**2*(data-1.))
bootstrapped_statistic = [test_statistic(bootstrap_resampling(data)) for t in range(10000)]

tenemos que su distribución muestral es

In [ ]:
bins, edges = np.histogram(bootstrapped_statistic, bins=20, density=True)
hv.Histogram((edges, bins), kdims='x', vdims='Density')

Usando esta distribución muestral podríamos implementar un test de hipótesis para nuestro estimador arbitrario

Por otro lado, si sólo nos interesa la incerteza del estimador podemos calcular un intervalo de confianza empírico con

In [ ]:
IC = np.percentile(bootstrapped_statistic, [2.5, 97.5])
print(f"Intervalo de confianza al 95%: {IC}")

:::{important}

El poder de nuestro computadores nos permite implementar *bootstrap* para obtener distribuciones muestrales de estadísticos arbitrarios sin necesidad de imponer supuestos para la distribución subyacente de los datos. 

:::

Usando bootstrap podemos calcular la distribución muestral de estadísticos como la mediana o la moda, que no tienen distribuciones asintóticas conocidas

:::{warning}

Sin embargo los datos deben ser iid para que bootstrap sea valido. Si los datos son dependientes (e.g. series de tiempo) se deben usar otros tipos de remuestreos como por ejemplo *block bootstrap*

:::

Para estudiar más a fondo las garantías de bootstrap recomiendo: http://www.stat.cmu.edu/~larry/=sml/Boot.pdf